In [2]:
import cv2
import numpy as np
from collections import deque

def setValues(x):
    print("")

    
points = [deque(maxlen = 1024)]
pointIndex = 0
    
radThreshold = 50
kernel = np.ones((5, 5), np.uint8)

paint = np.zeros((471, 636, 3)) + 255

cv2.namedWindow("Color Detection")
cv2.createTrackbar("Upper Hue", "Color Detection",
                   90, 180, setValues)
cv2.createTrackbar("Upper Saturation", "Color Detection",
                   255, 255, setValues)
cv2.createTrackbar("Upper Value", "Color Detection",
                   255, 255, setValues)
cv2.createTrackbar("Lower Hue", "Color Detection",
                   60, 180, setValues)
cv2.createTrackbar("Lower Saturation", "Color Detection",
                   72, 255, setValues)
cv2.createTrackbar("Lower Value", "Color Detection",
                   49, 255, setValues)

# Create videocapture object
cap = cv2.VideoCapture(0)
while True:
    
    # Read each frame from webcam
    success, cam = cap.read()
    cam = cv2.flip(cam, 1)
    hsv = cv2.cvtColor(cam, cv2.COLOR_BGR2HSV)
    
    upHue = cv2.getTrackbarPos("Upper Hue","Color Detection")
    upSat = cv2.getTrackbarPos("Upper Saturation","Color Detection")
    upVal = cv2.getTrackbarPos("Upper Value","Color Detection")
    lowHue = cv2.getTrackbarPos("Lower Hue","Color Detection")
    lowSat = cv2.getTrackbarPos("Lower Saturation","Color Detection")
    lowVal = cv2.getTrackbarPos("Lower Value","Color Detection")
    
    upHSV = np.array([upHue,upSat,upVal])
    lowHSV = np.array([lowHue,lowSat,lowVal])
    
    mask = cv2.inRange(hsv, lowHSV, upHSV)
    mask = cv2.erode(mask, kernel, iterations = 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center = None
    
    
    if(len(cnts) > 0):
        
        # Getting the center point
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        
        cv2.circle(cam, (int(x), int(y)), int(radius), (255, 255, 0) if (radius < radThreshold) else (0, 0, 255), 2)
    
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
        # Incrementing points
        points[pointIndex].appendleft(center)
    else:
        points.append(deque(maxlen = 512))
        pointIndex += 1
        
    # Drawing line
    for i in range(len(points)):
        for j in range(1,len(points[i])):
            if points[i][j-1] is None or points[i][j] is None:
                continue
            cv2.line(cam, points[i][j-1], points[i][j], (0,0,0), 2)
            cv2.line(paint, points[i][j-1], points[i][j], (0,0,0), 2)
    
    cv2.imshow("Camera", cam)
    cv2.imshow("Paint", paint)
    cv2.imshow("Mask", mask)
    
    # Open the OpenCV window until 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()